<a href="https://colab.research.google.com/github/Monika095/machine-learning-bootcamp/blob/main/supervised/01-basics/01-preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import sklearn

In [4]:
data = {
    'size': ['XL','L','M','L','M'],
    'color': ['red','green','blue','green','red'],
    'gender': ['f','m','m','f','f'],
    'price':[199.0, 89.0, 99.0, 129.0, 79.0],
    'weight': [500, 450, 300, 380, 410],
    'bought': ['yes', 'no', 'yes', 'no', 'yes']
    }
df_raw = pd.DataFrame(data=data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,f,199.0,500,yes
1,L,green,m,89.0,450,no
2,M,blue,m,99.0,300,yes
3,L,green,f,129.0,380,no
4,M,red,f,79.0,410,yes


In [5]:
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


In [6]:
for col in ['size','color','gender','bought']:
  df[col] = df[col].astype('category')

df['weight'] = df['weight'].astype('float')

In [7]:
df.describe()

,price,weight
count,5.000000,5.000000
mean,119.000000,408.000000
std,48.476799,75.299402
min,79.000000,300.000000
25%,89.000000,380.000000
50%,99.000000,410.000000
75%,129.000000,450.000000
max,199.000000,500.000000


In [8]:
df

,size,color,gender,price,weight,bought
0,XL,red,f,199.0,500.0,yes
1,L,green,m,89.0,450.0,no
2,M,blue,m,99.0,300.0,yes
3,L,green,f,129.0,380.0,no
4,M,red,f,79.0,410.0,yes


In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['bought'])
le.transform(df['bought'])


array([1, 0, 1, 0, 1])

In [10]:
le

LabelEncoder()

In [11]:
le.classes_

array(['no', 'yes'], dtype=object)

In [14]:
df['bought'] = le.transform(df['bought'])

TypeError: ignored

In [15]:
df

,size,color,gender,price,weight,bought
0,XL,red,f,199.0,500.0,1
1,L,green,m,89.0,450.0,0
2,M,blue,m,99.0,300.0,1
3,L,green,f,129.0,380.0,0
4,M,red,f,79.0,410.0,1


In [18]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoder.fit(df[['size']])

OneHotEncoder(sparse=False)

dodaje sparse- False bo inaczej jest macierz rzadka, czyli wiekszosc elementow to 0. zaoszczedzenie pamieci

In [19]:
encoder.transform(df[['size']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

Zakodowane M, L, XL. Żeby sprawdzić jak wyglądają kolumny dodaje "encoder.categories_"

In [21]:
encoder.categories_

[array(['L', 'M', 'XL'], dtype=object)]

chcemy miec 2 zmienne, zeby model zbyt nie uzaleznil sie od naszych zmiennych. mamy 3 ale wystarcza nam dwa, a trzecia mozna wywnioskowac z pozostałych, poniewaz nic nie jest nullem. Nie chcemy miec liniowej zaleznosci miedzy naszymi zmiennymi. dodajemy parametr drop

In [23]:
encoder = OneHotEncoder(drop = 'first', sparse = False)
encoder.fit(df[['size']])
encoder.transform(df[['size']])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

jesli mamy n zmiennych to po kodowaniu 0-1 mamy n-1 kolumn zakodowanych

Łatwiej używać do taich rzeczy pandas - get_dummies()

In [24]:
pd.get_dummies(data=df)

,price,weight,bought,size_L,size_M,size_XL,color_blue,color_green,color_red,gender_f,gender_m
0,199.0,500.0,1,0,0,1,0,0,1,1,0
1,89.0,450.0,0,1,0,0,0,1,0,0,1
2,99.0,300.0,1,0,1,0,1,0,0,0,1
3,129.0,380.0,0,1,0,0,0,1,0,1,0
4,79.0,410.0,1,0,1,0,0,0,1,1,0


In [25]:
pd.get_dummies(data=df, drop_first=True)

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_m
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


Stnadaryzacja!
std() - pandas nieobciązony
std() - numpy obciążony

In [26]:
from sklearn.preprocessing import scale

In [27]:
scale(df['price'])

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df[['price']])
scaler.transform(df[['price']])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [30]:
scaler = StandardScaler()
df[['price','weight']] = scaler.fit_transform(df[['price','weight']])
df

,size,color,gender,price,weight,bought
0,XL,red,f,1.845062,1.366002,1
1,L,green,m,-0.691898,0.623610,0
2,M,blue,m,-0.461266,-1.603567,1
3,L,green,f,0.230633,-0.415740,0
4,M,red,f,-0.922531,0.029696,1


In [31]:
df=df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,f,199.0,500,yes
1,L,green,m,89.0,450,no
2,M,blue,m,99.0,300,yes
3,L,green,f,129.0,380,no
4,M,red,f,79.0,410,yes


In [34]:
le= LabelEncoder()
df['bought'] = le.fit_transform(df['bought'])

df= pd.get_dummies(data = df, drop_first = True)

scaler = StandardScaler()
df[['price','weight']] = scaler.fit_transform(df[['price','weight']])


df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_m
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0
